In [ ]:
"""
Created on Mon Feb 26 16:10:50 2018

@author: HP
"""

import pandas as pd
import csv
import matplotlib.pyplot as plt
import re
import numpy as np
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models.keyedvectors import KeyedVectors
import tensorflow as tf



batchSize = 24
lstmUnits = 64
numClasses = 13
iterations = 100000
numDimensions = 100
maxSeqLength = 100

options = {
           'drug' :            [1,0,0,0,0,0,0,0,0,0,0,0,0],
           'hang' :            [0,1,0,0,0,0,0,0,0,0,0,0,0],
           'gun' :             [0,0,1,0,0,0,0,0,0,0,0,0,0],
           'alcohol' :         [0,0,0,1,0,0,0,0,0,0,0,0,0],
           'cut' :             [0,0,0,0,1,0,0,0,0,0,0,0,0],
           'jump' :            [0,0,0,0,0,1,0,0,0,0,0,0,0],
           'disease' :         [0,0,0,0,0,0,1,0,0,0,0,0,0],
           'bleed' :           [0,0,0,0,0,0,0,1,0,0,0,0,0],
           'drown' :           [0,0,0,0,0,0,0,0,1,0,0,0,0],
           'vehicle' :         [0,0,0,0,0,0,0,0,0,1,0,0,0],
           'carbon monoxide' : [0,0,0,0,0,0,0,0,0,0,1,0,0],
           'starve' :          [0,0,0,0,0,0,0,0,0,0,0,1,0],
           'electrocute' :     [0,0,0,0,0,0,0,0,0,0,0,0,1],
           }

########################################################################

strip_special_chars = re.compile("[^A-Za-z0-9 ]+")
def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())
########################################################################
def getLabel(S_method):
    #[drug,hang,gun,alcohol,cut,jump,disease,bleed,drown,vehicle,carbon monoxide,starve,electrocute]    
   
    return options[S_method]

#########################################################################

def loadGloveModel(gloveFile):
    glove2word2vec(glove_input_file = gloveFile, word2vec_output_file="gensim_glove_vectors_100.txt")
    glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors_100.txt", binary=False)
#    print("Loading Glove Model")
#    f = open(gloveFile,'r')
#    model = {}
#    for line in f:
#        splitLine = line.split()
#        word = splitLine[0]
#        embedding = np.array([float(val) for val in splitLine[1:]])
#        model[word] = embedding
#        print(word)
#    print("Done.",len(model)," words loaded!")
    return glove_model
##############################################################################
    
def convert_data_to_index(string_data, wv):
    index_data = []
    for word in string_data:
        if word in wv:
            index_data.append(wv[word].index)
    return index_data
###################################################################
    
def convertModelIntoEmbdMtrx(model,vector_dim):
    embedding_matrix = np.zeros((len(model.vocab), vector_dim),dtype=np.float32)
    for i in range(len(model.vocab)):
        embedding_vector = model.word_vec(model.index2word[i])
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

######################################################################

def splitTrainTest(frame):
    df_test = pd.DataFrame()
    df_train = pd.DataFrame()
    for k,v in options.items():
        tempbool = frame['S_method'] == k
        tempFrame = frame[tempbool]
        length = len(tempFrame)
        train, test = np.split(tempFrame, [int(.7*length)])    
        df_train = df_train.append(train)
        df_test = df_test.append(test)
    return df_train, df_test



#######################################################################
def getTrainBatch():
    labels = []
    arr = np.zeros([len(tndf.index), maxSeqLength])
    for i in range(len(tndf.index)):
        id_idx = tndf.index[i]
#        print("train Batch : " + str(id_idx))
        arr[i] = ids[id_idx]
        labels.append(tndf.label[tndf.index[i]])
    return arr, labels

def getTestBatch():
    labels = []
    arr = np.zeros([len(tsdf.index), maxSeqLength])
    for i in range(len(tsdf.index)):
        id_idx = tsdf.index[i]
        arr[i] = ids[id_idx]
        labels.append(tsdf.label[tsdf.index[i]])
    return arr, labels

rows = []
#csv_header = ['user', 'item', 'time', 'rating', 'review']
#frame_header = ['user', 'item', 'rating', 'review']

wordModel = loadGloveModel('../../Pretrained_WordVectors/glove.twitter.27B/glove.twitter.27B.100d.txt')
WVDic = wordModel.vocab
idx = convert_data_to_index("test string data".split(),wordModel.vocab)
wordVectors = convertModelIntoEmbdMtrx(wordModel,100)

ids = np.zeros((503, maxSeqLength), dtype='int32')
cmntCntr = 0
with open('Manual_Annotations_Sent.csv', 'rt') as f_input:
    for row in csv.DictReader(f_input, delimiter='|', fieldnames=['sent','S_method'], skipinitialspace=True):
        cleanedSent = cleanSentences(row['sent'])
        split = cleanedSent.split()
        wordCount = len(split)
        if wordCount <= maxSeqLength:
            idstemp = convert_data_to_index(split,wordModel.vocab)
            ids[cmntCntr][0:len(idstemp)] = idstemp
            rows.append([cleanedSent, getLabel(row['S_method']), row['S_method'], wordCount])
            cmntCntr = cmntCntr + 1
        
frame = pd.DataFrame(rows, columns=['sent','label','S_method','word_count'])
tndf, tsdf = splitTrainTest(frame)


tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [len(tndf.index), numClasses])
input_data = tf.placeholder(tf.int32, [len(tndf.index), maxSeqLength])

data = tf.Variable(tf.zeros([len(tndf.index), maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "./tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
with tf.Session() as sess:
    writer = tf.summary.FileWriter(logdir, sess.graph)



sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

for i in range(iterations):
    print("iteration " + str(i))
     #Next Batch of reviews
    nextBatch, nextBatchLabels = getTrainBatch();
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})

    #Write summary to Tensorboard
    if (i % 50 == 0):
        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
        writer.add_summary(summary, i)

#Save the network every 10,000 training iterations
    if (i % 1000 == 0 and i != 0):
        save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
        print("saved to %s" % save_path)
writer.close()